In [ ]:
import os
import pydicom
from pydicom.misc import is_dicom
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
import plotly.graph_objects as go
import nibabel as nib
import plotly.io as pio

In [ ]:
def load_dicom_images(directory, max_slices=None):
    slices = []
    for filename in sorted(os.listdir(directory)):
        if filename.endswith(".dcm"):
            path = os.path.join(directory, filename)
            dicom = pydicom.dcmread(path)
            slices.append(dicom)
            if max_slices and len(slices) >= max_slices:
                break
    # Ordenar as fatias por posição do paciente
    slices.sort(key=lambda x: x.ImagePositionPatient[2])
    return slices

def create_3d_image(slices):
    # Extrair arrays de pixel e empilhá-los
    pixel_arrays = [s.pixel_array for s in slices]
    image_3d = np.stack(pixel_arrays, axis=-1)
    # Converter para imagem ITK
    itk_image = sitk.GetImageFromArray(image_3d)
    return itk_image

def show_slices(image_array, num_slices=165, cols=11):
    # Calcular número de linhas necessárias
    rows = num_slices // cols + (num_slices % cols > 0)
    fig, axes = plt.subplots(rows, cols, figsize=(20, 20))

    for i in range(num_slices):
        row = i // cols
        col = i % cols
        axes[row, col].imshow(image_array[:, :, i], cmap='gray')
        axes[row, col].axis('off')
        axes[row, col].set_title(f'Slice {i}')

    # Remover subplots extras se o número de fatias for menor que o grid total
    for j in range(num_slices, rows * cols):
        fig.delaxes(axes.flatten()[j])

    plt.tight_layout()
    plt.show()

def correct_orientation(itk_image):
    # Obter a matriz de transformação atual
    origin = itk_image.GetOrigin()
    direction = itk_image.GetDirection()
    spacing = itk_image.GetSpacing()
    
    # Alterar a direção da imagem para que os eixos estejam orientados corretamente
    # Isso depende da orientação específica dos dados DICOM
    direction_corrected = [1, 0, 0, 0, 1, 0, 0, 0, -1]  # Exemplo para inverter o eixo Z
    
    itk_image.SetDirection(direction_corrected)
    
    # Atualizar a origem da imagem se necessário
    # Exemplo: modificar a origem para compensar a mudança na orientação
    origin_corrected = (origin[0], origin[1], origin[2] + (len(slices) - 1) * spacing[2])
    itk_image.SetOrigin(origin_corrected)
    
    return itk_image

# Caminho p DICOM
dicom_directory = "C:/Users/bruno/Desktop/IANS/mri_exemplo/DICOM/I60175"

# Carregar  DICOM
slices = load_dicom_images(dicom_directory, max_slices=166)  # Limitar a 166 imagens
image_3d = create_3d_image(slices)

# Corrigir a orientação da imagem
image_3d_corrected = correct_orientation(image_3d)

# Converter a imagem segmentada para um array NumPy para visualização
img_array = sitk.GetArrayFromImage(image_3d_corrected)

# Visualizar os cortes 2D da imagem segmentada (somente o cérebro)
show_slices(img_array, num_slices=166, cols=11)

# Salvar a imagem 3D segmentada como um arquivo NIfTI
sitk.WriteImage(image_3d, "C:/Users/bruno/Desktop/IANS/mri_exemplo/NIFTL/output_brain_2_oriented.nii.gz")

In [ ]:
import nibabel as nib
import matplotlib.pyplot as plt

# Carregar o arquivo .nii
nii_file = "C:/Users/bruno/Desktop/IANS/mri_exemplo/NIFTL/output_brain.nii.gz"
img = nib.load(nii_file)
data = img.get_fdata()

# Exibir informações sobre a imagem
print("Shape da imagem:", data.shape)
print("Tipo de dados:", data.dtype)

# Configurar o tamanho da figura
plt.figure(figsize=(18, 6))

# Visualizar um corte sagital (lateral) com rotação
slice_index_sagital = data.shape[0] // 2  # Meio do eixo x
sagital_slice = data[slice_index_sagital, :, :]
plt.subplot(1, 3, 1)
plt.imshow(np.rot90(sagital_slice, k=-1), cmap='gray')  # Rotacionar 90 graus
plt.title('Corte Sagital')

# Visualizar um corte coronal (frontal) com rotação
slice_index_coronal = data.shape[1] // 2  # Meio do eixo y
coronal_slice = data[:, slice_index_coronal, :]
plt.subplot(1, 3, 2)
plt.imshow(np.rot90(coronal_slice, k=-1), cmap='gray')  # Rotacionar 90 graus
plt.title('Corte Coronal')

# Visualizar um corte axial (horizontal) com rotação
slice_index_axial = data.shape[2] // 2  # Meio do eixo z
axial_slice = data[:, :, slice_index_axial]
plt.subplot(1, 3, 3)
plt.imshow(axial_slice, cmap='gray')  # Rotacionar 90 graus
plt.title('Corte Axial')

# Exibir a figura com os três cortes
plt.tight_layout()
plt.show()


In [10]:
import os

def nome_arq(arq):
    name = 'ADNI_' + arq
    name = name.split('.')[0]
    return name

#Código para associar arquivos NIFTL com seus XML
DIR_BASE = os.path.abspath('/mnt/d/ADNI/ADNI1')
DIR_NII = os.path.abspath(os.path.join(DIR_BASE, 'ADNI_nii_raw'))
DIR_XML = os.path.abspath(os.path.join(DIR_BASE, 'ADNI1_Screening_metadata', 'ADNI'))

for item in os.listdir(DIR_NII):
    name = nome_arq(item)
    for xismele in os.listdir(DIR_XML):
        if xismele.startswith(name):
            print(f"RECEBA {xismele}")

RECEBA ADNI_002_S_0295_MP-RAGE_S13408_I13722.xml
RECEBA ADNI_002_S_0782_MP-RAGE_S17835_I20519.xml
RECEBA ADNI_002_S_1155_MP-RAGE_S24144_I33393.xml
RECEBA ADNI_002_S_1261_MP-RAGE_S26574_I40503.xml
RECEBA ADNI_005_S_0221_MP-RAGE_S11956_I11604.xml
RECEBA ADNI_005_S_0221_MP-RAGE_S11957_I11605.xml
RECEBA ADNI_005_S_0221_MP-RAGE_S11958_I11606.xml
RECEBA ADNI_005_S_0222_MP-RAGE_S11753_I11299.xml
RECEBA ADNI_005_S_0222_MP-RAGE_S11754_I11300.xml
RECEBA ADNI_005_S_0324_MP-RAGE_S12600_I12642.xml
RECEBA ADNI_005_S_1224_MP-RAGE_S25412_I37284.xml
RECEBA ADNI_006_S_0322_MP-RAGE_S15659_I17197.xml
RECEBA ADNI_006_S_0484_MP-RAGE_S15774_I17377.xml
RECEBA ADNI_006_S_0484_MP-RAGE_S18837_I24264.xml
RECEBA ADNI_006_S_0498_MP-RAGE_S15857_I17505.xml
RECEBA ADNI_006_S_0498_MP-RAGE_S19762_I25790.xml
RECEBA ADNI_006_S_0653_MP-RAGE_S16073_I17867.xml
RECEBA ADNI_006_S_0653_MP-RAGE_S23255_I31252.xml
RECEBA ADNI_006_S_0731_MP-RAGE_S18321_I23468.xml
RECEBA ADNI_006_S_1130_MP-RAGE_S23456_I31799.xml
RECEBA ADNI_007_S_00